In [52]:
import os

# Define the folder
folder_path = 'PERFILES2'

# Check if folder exists
if not os.path.exists(folder_path):
    print(f"Error: Folder '{folder_path}' does not exist.")
    exit()

# Delete all .pdf files in PERFILES2
pdf_count = 0
for file_name in os.listdir(folder_path):
    if file_name.endswith('.pdf'):
        file_path = os.path.join(folder_path, file_name)
        try:
            os.remove(file_path)
            print(f"Deleted: {file_path}")
            pdf_count += 1
        except Exception as e:
            print(f"Error deleting {file_path}: {e}")

print(f"Deleted {pdf_count} PDF file(s) from {folder_path}.")

Deleted 0 PDF file(s) from PERFILES2.


In [53]:
def parse_dat_file(file_path, max_rpm_count=15):
    rpm_data = {}
    current_rpm = None
    data_lines = []
    rpm_count = 0
    required_columns = ['V', 'J', 'Pe', 'Ct', 'Cp', 'PWR_W', 'Torque_Nm', 'Thrust_N', 'THR_PWR', 'FOM']
    
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            for line in lines:
                line = line.strip()
                if not line:
                    continue
                if line.startswith('PROP RPM'):
                    if current_rpm is not None and data_lines and rpm_count < max_rpm_count:
                        df = pd.DataFrame(data_lines, columns=['V', 'J', 'Pe', 'Ct', 'Cp', 'PWR_Hp', 'Torque_InLbf', 
                                                              'Thrust_Lbf', 'PWR_W', 'Torque_Nm', 'Thrust_N', 
                                                              'THR_PWR', 'Mach', 'Reyn', 'FOM'])
                        # Verify required columns
                        if not all(col in df.columns for col in required_columns):
                            missing = [col for col in required_columns if col not in df.columns]
                            raise ValueError(f"Missing columns: {missing}")
                        df = df[required_columns].astype(float).head(15)
                        rpm_data[current_rpm] = df
                        data_lines = []
                    match = re.search(r'PROP RPM\s*=\s*(\d+)', line)
                    if match and rpm_count < max_rpm_count:
                        current_rpm = int(match.group(1))
                        rpm_count += 1
                elif current_rpm is not None and not line.startswith('(') and not line.startswith('V') and rpm_count <= max_rpm_count:
                    parts = line.split()
                    if len(parts) >= 14:
                        data_lines.append(parts)
            
            if current_rpm is not None and data_lines and rpm_count <= max_rpm_count:
                df = pd.DataFrame(data_lines, columns=['V', 'J', 'Pe', 'Ct', 'Cp', 'PWR_Hp', 'Torque_InLbf', 
                                                      'Thrust_Lbf', 'PWR_W', 'Torque_Nm', 'Thrust_N', 
                                                      'THR_PWR', 'Mach', 'Reyn', 'FOM'])
                if not all(col in df.columns for col in required_columns):
                    missing = [col for col in required_columns if col not in df.columns]
                    raise ValueError(f"Missing columns: {missing}")
                df = df[required_columns].astype(float).head(15)
                rpm_data[current_rpm] = df
    except Exception as e:
        print(f"Error parsing {file_path}: {e}")
        return None
    
    return rpm_data

In [54]:
# Define folders and parameters
input_folder = 'PERFILES2'
output_folder = 'performance_graphs'
parameters = ['J', 'Pe', 'Ct', 'Cp', 'PWR_W', 'Torque_Nm', 'Thrust_N', 'THR_PWR', 'FOM']
param_labels = {
    'J': 'Advance Ratio (J)', 'Pe': 'Efficiency (Pe)', 'Ct': 'Thrust Coefficient (Ct)', 
    'Cp': 'Power Coefficient (Cp)', 'PWR_W': 'Power (W)', 'Torque_Nm': 'Torque (Nm)', 
    'Thrust_N': 'Thrust (N)', 'THR_PWR': 'Thrust-to-Power Ratio (g/W)', 
    'FOM': 'Figure of Merit (FOM)'
}

# Colors and line styles for 15 RPMs
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', 
          '#7f7f7f', '#bcbd22', '#17becf', '#1b9e77', '#d95f02', '#7570b3', '#e7298a', '#66a61e']
line_styles = ['-', '--', ':', '-.'] * 4

# Check if input folder exists
if not os.path.exists(input_folder):
    print(f"Error: Input folder '{input_folder}' does not exist. Please create it and add .dat files.")
    exit()

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Process each .dat file
for file_name in os.listdir(input_folder):
    if file_name.endswith('.dat'):
        file_path = os.path.join(input_folder, file_name)
        print(f"Processing {file_name}...")
        
        # Parse data
        rpm_data = parse_dat_file(file_path, max_rpm_count=15)
        if rpm_data is None:
            print(f"Skipping {file_name} due to parsing error.")
            continue
        
        # Create a single figure with a 3x3 grid for 9 plots
        n_params = len(parameters)
        n_cols = 3
        n_rows = 3
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 12), sharex=False, sharey=True)
        axes = axes.flatten()
        
        # Generate plots
        for i, param in enumerate(parameters):
            ax = axes[i]
            for j, (rpm, df) in enumerate(sorted(rpm_data.items())):
                ax.plot(df[param], df['V'], label=f'RPM {rpm}', color=colors[j % len(colors)], 
                        linestyle=line_styles[j % len(line_styles)], linewidth=1.5, marker='o', markersize=3, alpha=0.8)
            
            ax.set_xlabel(param_labels[param], fontsize=8)
            ax.set_ylabel('Velocity (mph)', fontsize=8)
            ax.set_title(f'V vs. {param_labels[param]}', fontsize=9)
            ax.grid(True, linestyle='--', alpha=0.7)
            ax.legend(loc='best', fontsize=6, frameon=True, framealpha=0.9, edgecolor='black')
            ax.margins(x=0.05, y=0.05)
        
        # Adjust layout and style
        plt.style.use('ggplot')
        plt.suptitle(f'Propeller Performance: {file_name[:-4]}', fontsize=14, y=0.98)
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        
        # Save to PDF in performance_graphs
        pdf_path = os.path.join(output_folder, f'{file_name[:-4]}.pdf')
        with PdfPages(pdf_path) as pdf:
            pdf.savefig(dpi=300, bbox_inches='tight')
        
        plt.close(fig)
        print(f"Saved PDF: {pdf_path}")

Processing PER3_105x45.dat...
Saved PDF: performance_graphs\PER3_105x45.pdf
Processing PER3_105x6.dat...
Saved PDF: performance_graphs\PER3_105x6.pdf
Processing PER3_10x10.dat...
Saved PDF: performance_graphs\PER3_10x10.pdf
Processing PER3_10x10E.dat...
Saved PDF: performance_graphs\PER3_10x10E.pdf
Processing PER3_10x12WE.dat...
Saved PDF: performance_graphs\PER3_10x12WE.pdf
Processing PER3_10x3.dat...
Saved PDF: performance_graphs\PER3_10x3.pdf
Processing PER3_10x38MRF-RH.dat...
Saved PDF: performance_graphs\PER3_10x38MRF-RH.pdf
Processing PER3_10x38SF.dat...
Saved PDF: performance_graphs\PER3_10x38SF.pdf
Processing PER3_10x4.dat...
Saved PDF: performance_graphs\PER3_10x4.pdf
Processing PER3_10x45MR.dat...
Saved PDF: performance_graphs\PER3_10x45MR.pdf
Processing PER3_10x45MRF-RH.dat...
Saved PDF: performance_graphs\PER3_10x45MRF-RH.pdf
Processing PER3_10x46SF.dat...
Saved PDF: performance_graphs\PER3_10x46SF.pdf
Processing PER3_10x47SF.dat...
Saved PDF: performance_graphs\PER3_10x47S

In [56]:
import os

# Define the folder path
folder_path = 'performance_hraph'  # Replace with 'performance_graphs' or another folder as needed

# Check if folder exists
if not os.path.exists(folder_path):
    print(f"Error: Folder '{folder_path}' does not exist.")
    exit()

# Count all files
all_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
print(f"Total number of files in {folder_path}: {len(all_files)}")

# Optionally, count specific file types (e.g., .dat or .pdf)
dat_files = [f for f in os.listdir(folder_path) if f.endswith('.dat') and os.path.isfile(os.path.join(folder_path, f))]
pdf_files = [f for f in os.listdir(folder_path) if f.endswith('.pdf') and os.path.isfile(os.path.join(folder_path, f))]

print(f"Number of .dat files in {folder_path}: {len(dat_files)}")
print(f"Number of .pdf files in {folder_path}: {len(pdf_files)}")

Total number of files in performance_graphs: 437
Number of .dat files in performance_graphs: 0
Number of .pdf files in performance_graphs: 437
